In [127]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import networkx as nx
import plotly.graph_objects as go
import ast
import requests as r
from io import StringIO

pd.set_option('display.max_columns', 50)

# Introdução

Em nossas análises, temos como meta principal o mapeamento dos jogos com maior taxa de engajamento.

Nesse sentido, após analisar as variáveis, decidimos por eliminar algumas delas:


'playerage' - Nos parece ter alguns problemas de formatação (problemas com a tabela ascii) e entrega um faixa de idade recomendada pelos jogadores. Consideramos muito subjetivo e prefirimos usar a 'minage', apesar de serem valores próximos na média;

'gamelink' - Se quisermos ver o jogo no BGG, basta pesquisá-lo por ser nome;

'boardgameexpansion' - Cada jogo tem a sua específica, não faz sentido analisar;

'max_community' (Fornecido pela comunidade) - No geral, igual ao 'maxplayers' (fornecido pelos fabricantes), só que com alguns valores faltando; Em geral, dados fornecidos pelo público podem conter problemas, como falta de votos em certo jogo, mas no geral são mais confiáveis.

'minplayers' - Fornecido pelos fabricantes, indicam o limite de jogabilidade, mas não garante uma experiência interessante. Sendo assim, vale usar a opinião da comunidade;

In [128]:
def faz_df(url):
    req = r.request('GET', url, headers={'user-agent': 'Mozilla/5.0'})
    df = pd.read_json(StringIO(req.text), encoding='UTF-8')
    return df

In [129]:
# Comentar
information_df = faz_df('https://treinamento-ndados.s3.sa-east-1.amazonaws.com/boardgames_information_raw.json')

In [130]:
df = pd.read_excel('df_unificado.xlsx')
df['minplayers'] = information_df['minplayers'].copy()

Após removermos as variáveis que eram explicitamente descartáveis, devemos remover as que contém dados aparentemente ruins e arrumar as que têm má formatação

Vou preencher os valores nulos de 'min_community' com 'minplayers' + 2. Prefirimo os 'min_community' pela maior confiabilidade das opiniões dos avaliadores do BGG.

In [133]:
#decidimos por subistituir os valores nulos de 'min_community' por 'minplayers' + 2
df['min_community'].fillna(df['minplayers']+2, inplace=True)
df = df.drop("minplayers", axis=1)

Encontramos um problema: há diversos dados nulos nas colunas baverage e average, e não sabemos o que essa nulicidade representa diretamente. Portanto, vamos prever esses valores com base na razão entre average e baverage. A seguir fica o passo a passo de como fizemos o cálculo da razão.

Primeiramente, iremos tratar as linhas com nenhuma avaliação de jogadores. Ao fazer isso, notamos que essas linhas não contem dados releveantes em nenhuma outra coluna, como mostrado abaixo.

In [134]:
# Criando um novo DataFrame para manipulação
df_manipulado = df.copy()

Vou comparar os valores de cada variável que contém userrated == 0 com o valor total delas, para saber sua representatividade.

In [135]:
def verifica_porcentagem(df:pd.DataFrame, coluna):
    valor = df.query('usersrated == 0').sum()[coluna]
    total = df[coluna].sum()
    return valor / total * 100

In [136]:
colunas_analisadas = ['numweights','numgeeklists', 'numwanting', 'numcomments', 'siteviews', 'blogs', 'weblink']
for coluna in colunas_analisadas:
    print(f"A porcentagem de {coluna} é {verifica_porcentagem(df_manipulado, coluna)}")

A porcentagem de numweights é 0.00031196251458424755
A porcentagem de numgeeklists é 0.011253797901023462
A porcentagem de numwanting é 0.007868988949147294
A porcentagem de numcomments é 0.0009131586156515387
A porcentagem de siteviews é 0.02870729860782448
A porcentagem de blogs é 0.0004923682914820286
A porcentagem de weblink é 0.009585812996764788


Como todos os valores representam nem 1% do total, serão desconsiderados.

In [137]:
# Removendo todas as linhas com usersrated == 0
df_manipulado = df_manipulado.query('usersrated != 0').copy()

Analisando baverage == 0

In [138]:
print(f"A porcentagem de usersrated com baverage == 0 por total é {df_manipulado.query('baverage == 0')['usersrated'].sum() / df_manipulado['usersrated'].sum() * 100}")

A porcentagem de usersrated com baverage == 0 por total é 0.10620128769908506


Como as ocorrencias de baverage == 0 refletem 0.1% dos votos todais, optei por desonsiderá-las para efeito de cálculo

In [139]:
df_manipulado = df_manipulado.query('baverage != 0').copy()

Lidando com dados nulos

Como os dados nulos ocasionarão em futuros erros ou novos dados nulos após efetuação do cálculo, desconsideraremos

In [140]:
df_manipulado = df_manipulado.dropna().copy()

Cálculo da razão

In [141]:
df_manipulado['average/baverage'] = df_manipulado['average'] / df_manipulado['baverage']
df_manipulado['average/baverage_ponderado'] =df_manipulado['average/baverage'] * df_manipulado['usersrated']
razao_media = df_manipulado['average/baverage_ponderado'].sum() / df_manipulado['usersrated'].sum()

#### Remapeando o DataFrame original

 Removendo registros sem votos e tratando os dados nulos e zerados como o mesmo problema:

In [142]:
df = df.query('usersrated != 0').copy()
df = df.fillna(0)

 Removendo duplo 0 (average == 0 e baverage == 0)

In [143]:
df['average+baverage'] = df['average'] + df['baverage']
df = df.query('average+baverage != 0').copy()
df.drop(labels=['average+baverage'], axis=1, inplace=True)

Efetuando o mapeamento

In [144]:
# Defini duas funções para tratar os dados zerados(que antes eram nulos) das colunas
# average e baverage

def tratamento_average(linha):
    if linha['average'] <= 0.1:
        return linha['baverage'] * razao_media
    else:
        return linha['average']
    
def tratamento_baverage(linha):
    if linha['baverage'] <= 0.1:
        return linha['average'] / razao_media
    else:
        return linha['baverage']

In [145]:
# Atualizando a coluna average
df['average'] = df.apply(tratamento_average, axis=1)
# Atualizando a coluna baverage
df['baverage'] = df.apply(tratamento_baverage, axis=1)

A partir daqui, o nosso DF não possui mais nenhuma coluna com valores nulos ou desformatados. Sendo assim, devemos encontrar algum método de medir a popularidade de cada jogo, para assim descobrirmos quais categorias, mecanicas e familias de jogos são mais recorrentes nas vendas. Para isso, vamos utilizar um método de machine learning chamado clusterização, o qual irá agrupar jogos similares em um mesmo 'cluster', com base em suas características como nota, total de avaliações, visitações em sua página no site, etc.

Com essa tecnica, portanto, poderemos juntar jogos mais populares e, posteriormente, analisar quais de suas características os tornam mais famosos.

## Machine Learning: clusterização dos dados estatísticos

Antes de aplicarmos o algoritmo em si, devemos entender o motivo pelo qual ele funcionará para a nossa análise. A princípio, queremos descobrir o perfil dos jogos mais populares para então recomendar outros com o mesmo perfil, mas que não é tão popular por motivos como: lançamento recente, campanha de marketing da desenvolvedora não tão satisfatória, poucas recomendações, etc. Para isso, a clusterização irá agrupar os jogos de acordo com seus dados estatísticos, como nota e quantidade de avaliações, para então podermos analisar as características do grupo mais popular.

Antes de começar, vamos selecionar as colunas que medem, estatísticamente, a popularidade/qualidade de um jogo:

Quantidade de avaliações,  quantidade de listas geek que o jogo está incluso, quantidade de pessoas que demonstram querer o jogo, quantidade de comentários sobre o jogo no site, quantiadde de visualizações na página do jogo no site, quantidade de citações em notícias, blogs, sites e podcasts, média de avaliação do jogo segundo os jogadores e o site BGG.

##### Usarei o algoritmo de clusterização Kmeans

In [146]:
from sklearn.cluster import KMeans
from sklearn import metrics

Agrupei jogos em 5 grupos com base em suas estatísticas de popularidade. Assim, ao indentificaros os melhores grupos, poderemos analisar quais de suas características categóricas os fazem tão populares e, então, recomendar jogos com as mesmas características.

In [147]:
colunas_clusterizadas = ['usersrated', 'numgeeklists', 'numwanting', 'numcomments', 
                         'siteviews', 'news', 'blogs', 'weblink', 'podcast', 'average',
                          'baverage']
df_clusterizado = df[colunas_clusterizadas].copy()
model_stat = KMeans(n_clusters=5, random_state=0, n_init='auto')
labels_stat = model_stat.fit_predict(df_clusterizado)
df_clusterizado['Cluster'] = labels_stat

Descrição dos clusters:


In [148]:
# Função que agrupa o dataframe clusterizado e cria a coluna com a quantidade total de itens em cada cluster
def descricao(df: pd.DataFrame, grupamento: str) -> pd.DataFrame:
    descricao = df.groupby(grupamento)
    n = descricao.size()
    descricao = descricao.mean()
    descricao['Quantidade Total'] = n
    
    return descricao

In [149]:
descricao(df_clusterizado, 'Cluster')

,usersrated,numgeeklists,numwanting,numcomments,siteviews,news,blogs,weblink,podcast,average,baverage,Quantidade Total
Cluster,,,,,,,,,,,,
0,222.489819,108.769222,16.329698,76.928563,3.276307e+04,0.339769,3.768610,4.216090,0.701458,6.216062,5.553143,17974
1,10955.879365,2629.755556,480.533333,2351.187302,8.368732e+05,3.612698,130.158730,41.850794,24.307937,7.381665,7.011795,315
2,39694.172414,7448.620690,956.241379,7306.068966,3.858579e+06,8.896552,526.068966,94.206897,100.137931,7.864858,7.666282,29
3,2873.803818,1025.134957,166.907176,766.382488,2.793917e+05,1.615537,39.330481,18.776169,7.949967,7.022249,6.332230,1519
4,22617.531646,4172.139241,780.000000,4212.658228,1.745673e+06,5.278481,245.050633,62.379747,46.126582,7.610450,7.345284,79


Para indentificar qual cluster contém os melhores jogos, farei um gráfico comparando a nota média da BGG de cada cluster, e outro com a quantidade de avaliações média.

In [150]:
agrupamento_cluster = df_clusterizado.groupby(by=['Cluster']).mean()
fig_nota_cluster = px.bar(agrupamento_cluster, x=agrupamento_cluster.index, y='baverage', 
                          title='Nota média dos jogos em cada cluster').update_layout(
                            yaxis_title='Nota média da BGG'
                          ).update_traces(marker_color='purple')
fig_nota_cluster.show()

In [151]:
fig_avaliacoes_cluster = px.bar(agrupamento_cluster, x=agrupamento_cluster.index, y='usersrated', 
                                title='Quantidade de avaliações média de cada cluster').update_layout(
                                    yaxis_title='Quantidade de avaliações'
                                ).update_traces(marker_color='purple')
fig_avaliacoes_cluster.show()

#### Explicação de cada cluster

#### Visualizando os clusters:

In [152]:
# Vamos visualizar os clusters de acordo com as seguintes caracteristicas: Quantidade de avaliações(usersrated), visualizações no site(siteviews)
# e nota (baverage_tratado)
fig_cluster = px.scatter_3d(df_clusterizado,x='usersrated', y='siteviews', z='baverage', color=df_clusterizado['Cluster'].astype(str), title='Distribuição dos jogos por cluster')
fig_cluster.update_layout(scene=dict(
                                    xaxis_title='Qnt Avaliações',
                                    yaxis_title='Visitações',
                                    zaxis_title='Nota'))
fig_cluster.show()

Cluster 0 - Por mais que seja um grup de jogos aparentemente ruim, por ter pouquíssimas avaliações e votos, há ampla gama de jogos, estes distintos entre si. Há jogos muito bem votados e jogos avaliados de forma péssima. Acreditamos não ser o cluster ideal para se avaliar e buscar nossos jogos-alvo, mas pode ser útil em alguma análise adicional de mapeamento.

Cluster 1 e 3 -  Muito parecidos entre si. Julgamos serem os menos importanes, pois são piores que os clusters 2 e 4 em todos os aspectos, sem muito desvio em relação a média. Logo, os deixaremos de lado.

Cluster 2 - O nosso melhor cluster, em todos os aspectos, extremamente nichado. Seu único defeito, do ponto de vista argumentativo, é o pequeno tamanho da amostra (pouco mais de 20 jogos), mas que melhora a visualização de alguns gráficos.

Cluster 4 - Extremamente similar ao cluster 2 em todos os aspectos, levemente pior, max com muito mais elementos. Interessante de se explorar.

Podemos concluir, portanto, que o cluster 2 e 4 são os mais bem avaliados e possuem a maior quantidade de avaliações. Nesse sentido, a partir de agora iremos avaliar qual as características dos jogos nesses clusters os fazem ser tão bem avaliados/famosos. 

Entretanto, vale ressaltar que os jogos do cluster 0, 1 e 3 não serão desconsiderados, eles serão usados posteriormente como sugestões que contém as mesmas características dos jogos dos clusters mais populares, mas que não atingem a qualidade/popularidade por conta de outros fatores, como um recente lançamento, por exemplo.

In [153]:
df['cluster'] = df_clusterizado['Cluster'].copy()

# Análise de características

Para a análise fazer sentido, vamos descartar todas as colunas que contem informações específicas de apenas um jogo, como quais e quantas expansões ele tem, e também dados como 'quais/quantos premios determinado jogo ganhou', visto que não dizem nada sobre o jogo em si. Além disso, descartamos categorias quantitativas irrelevantes como 'quantidade de mecânicas que o jogo tem' dado que o importante é qual mecânica o jogo possui, e não quantas.

Portanto, as características selecionadas foram: categorias, mecânicas e familias que os jogos pertencem.

É importante ressaltar, entretanto, que descartar essas informações agora não significa jogá-las no lixo. Posteriormente, elas podem ser usadas para o mapeamento dos jogos em grafos ou indentificação dos publicadores que mais lançam jogos com as categorias selecionadas aqui.

In [154]:
df_filtrado = df.drop(labels=['boardgamedesigner', 'boardgameartist', 'boardgamepublisher', 'boardgamehonor'
                      ], axis=1)

Primeiramente, vamos transformar os dados das categorias em listas.

In [155]:
# Aqui transformei o tipo dos valores dessas colunas de str para listas
df_filtrado['boardgamecategory'] = df_filtrado.boardgamecategory.apply(lambda x: x[1:-1].split(','))
df_filtrado['boardgamemechanic'] = df_filtrado.boardgamemechanic.apply(lambda x: x[1:-1].split(','))
df_filtrado['boardgamefamily'] = df_filtrado.boardgamefamily.apply(lambda x: x[1:-1].split(','))

In [156]:
# Defini uma função para formatar a lista que foi transformada na célula anterior
def formata_lista(lista_teste):
    lista_formatada = []
    for elemento in lista_teste:
        elemento_formatado = elemento.strip().replace("'", "")
        lista_formatada.append(elemento_formatado)
    return lista_formatada

In [157]:
# Formatando as listas de cada colunas
df_filtrado['boardgamecategory'] = df_filtrado.boardgamecategory.apply(formata_lista)
df_filtrado['boardgamemechanic'] = df_filtrado.boardgamemechanic.apply(formata_lista)
df_filtrado['boardgamefamily'] = df_filtrado.boardgamefamily.apply(formata_lista)

Criarei um DataFrame com apenas os jogos dos clusters 2 e 4.

In [158]:
df_filtrado_cluster2e4 = df_filtrado.query('cluster == 2 | cluster == 4').copy()

#### Idade:

Agora, vamos analisar a idade mínima predominante nesses jogos:

In [187]:
#criando as classes de idade
for index, row in df_filtrado_cluster2e4.iterrows():
    if row['minage'] <= 6:
        df_filtrado_cluster2e4.loc[index,'age_class'] =  'early_child'
    if row['minage'] >= 7:
        df_filtrado_cluster2e4.loc[index,'age_class'] =  'grown_child'
    if row['minage'] >= 12:
        df_filtrado_cluster2e4.loc[index,'age_class'] =  'teenager'
    if row['minage'] >= 15:
        df_filtrado_cluster2e4.loc[index,'age_class'] =  'adult' 

In [229]:
fig = px.scatter(df_filtrado_cluster2e4, x="yearpublished", y="usersrated", color="age_class",
                 size="boardgamehonor_cnt")
fig.update_xaxes(range=[2000, 2020])
fig.show()

Vemos que a maior parte dos jogos estão no grupo teenager e grown_child, ou seja, os jogos mais populares tem idade mínima entre 12 e 15 anos. Nesse sentido, indentificamos mais uma característica dos jogos populares. 

#### Dados categóricos

Vamos analisar agora os dados categóricos dos jogos dos clusters 2 e 4. Para assim podermos criar um perfil de jogo que vale a pena ser investido recursos sobre.

Como cada jogo pode ter mais de uma mecanica e pertencer a mais de uma categoria/familia, defini uma função que conta quantas aparições cada valor unico de mecanica, categoria e familia aparecem. Com isso, poderemos analisar cada feature individualmente e extrair quais são mais populares.

Além disso, como podem existir mais categorias que mecanicas, por exemplo, então defini uma função para analisar apenas o valor percentual de determinado feature em relação ao total de aparições. Assim, os gráficos ficarão melhores de visualizar.

In [159]:
# Função que retorna um data frame com a contagem que cada feature unico de um DataFrame
def conta_feature(df: pd.DataFrame, coluna) -> pd.DataFrame:
    count_df = {}
    for lista in df[coluna]:
        for valor in lista:
            count_df[valor] = count_df.get(valor, 0) + 1
    count_df = pd.DataFrame.from_dict(count_df, orient='index', columns=['contagem'])
    count_df.sort_values(by=['contagem'], ascending=False, inplace=True)
    count_df.reset_index(names='feature', inplace=True)
    return count_df

# Função que retorna a porcentagem de cada item de uma coluna de um DataFrame
def cria_coluna_porcentagem(df: pd.DataFrame, coluna) -> pd.DataFrame:
    soma = df[coluna].sum()
    df['porcentagem'] = df[coluna] / soma * 100
    return df

### Categorias:

Vamos ver quais categorias mais aparecem nos jogos dos clusters 2 e 4.

In [160]:
# Criando um DataFrame com a contagem de cada categoria
category_count2e4 = conta_feature(df_filtrado_cluster2e4, 'boardgamecategory')
# Criando coluna de porcentagem:
category_count2e4 = cria_coluna_porcentagem(category_count2e4, 'contagem')

In [161]:
fig_categoria = px.bar(category_count2e4.head(10), x='feature', y='porcentagem',
                        title='Porcentegem de aparição de cada categoria').update_layout(
                            xaxis_title='Categoria', yaxis_title='Porcentagem'
                        ).update_traces(marker_color='purple')
fig_categoria.show()

Aqui temos as 10 categorias que mais aparecem nos jogos considerados mais populares. Selecionei 10 pois elas somadas representam aproximadamente 50% de quantidade de aparições totais.

Avaliando as 3 primeiras categorias, aonde acreditamos que deve ser depositado foco, vemos que o destaque são para aquelas que desencadeiam emoções no público: Fantasia, luta e aventura. Em linha com a análise etária mínima dos jogadores, vemos que o público que concede popularidade e prêmios aos jogos tem por volta de 10-13 anos.

Categorias como 'fantasia' e 'aventura', por exemplo, são bastante amplas, e por conta disso vamos analisar também as mecânicas e famílias que os jogos pertencem. Além disso, elas duas e 'Fighting' representam quase 25% de todas as aparições, o que as tornam os principais indicadores do que torna um jogo popular.

Outras categorias como 'World war 2' ou 'Trains', são bem específicas, e refletem jogos com temas nichados, e por conta disso representam pouco no total. Logo, não serão usadas como categorias relevantes para o perfil de jogo popular.

### Mecânicas

In [162]:
# Criando um DataFrame com a contagem de cada mecânica
mechanic_count2e4 = conta_feature(df_filtrado_cluster2e4, 'boardgamemechanic')
# Criando coluna de porcentagem:
mechanic_count2e4 = cria_coluna_porcentagem(mechanic_count2e4, 'contagem')

In [163]:
fig_mecanica = px.bar(mechanic_count2e4.head(11), x='feature', y='porcentagem',
                        title='Porcentegem de aparição de cada mecânica').update_layout(
                            xaxis_title='Mecânica', yaxis_title='Porcentagem'
                        ).update_traces(marker_color='purple')
fig_mecanica.show()

Fica claro que nichar um jogo pode ser muito interessante para certos grupos de pessoas, mas os jogos mais presentes no cluster principal são os que possoem mecânicas mais simples. Prezar pela simplicidade não só é uma característica conservadora do ponto de vista de investimento, como também é o que atinge o público em potêncial.

Como no caso das categorias, aqui colocamos as 11 mecânicas que mais aparecem, pois representam 50% da contagem geral de aparições.

Vemos que mecânicas como 'Dice Rolling' e 'Hand Management' aparecem muito, representando 17% do total de aparições.

### Famílias

As famílias se demonstraram como um caso um pouco diferente. Existem diversas famílias e muitas delas dizem a mesma coisa que as categorias, como a família 'Components: Miniature' representa a mesma coisa que a categoria 'Miniatures'. Por conta disso, as famílias servirão como indicador secundário no momento de avaliar as características dos jogos populares.

In [164]:
# Criando um DataFrame com a contagem de cada mecânica
family_count2e4 = conta_feature(df_filtrado_cluster2e4, 'boardgamefamily')
# Criando coluna de porcentagem:
family_count2e4 = cria_coluna_porcentagem(family_count2e4, 'contagem')

In [165]:
fig_familia = px.bar(family_count2e4.head(10), x='feature', y='porcentagem',
                        title='Porcentegem de aparição de cada família').update_layout(
                            xaxis_title='Família', yaxis_title='Porcentagem'
                        ).update_traces(marker_color='purple')
fig_familia.show()

## Determinando jogos interessantes

Agora que conseguimos indentificar as características dos games mais famosos, vamos verificar quais jogos de outros clusters se encaixam nesse perfil de popularidade. Com isso, poderemos indentificar quais jogos tem potencial para se tornarem famosos, e merecem uma estratégia de divulgação própria.

In [166]:
def conta_lista(lista: list):
    contador = 0
    for item in lista:
        contador += 1
    return contador

df_filtrado['contagem de categorias'] = df_filtrado['boardgamecategory'].apply(conta_lista)
df_filtrado['contagem de mecanicas'] = df_filtrado['boardgamemechanic'].apply(conta_lista)

max_number_categ = df_filtrado['contagem de categorias'].max()
max_number_mechanic = df_filtrado['contagem de mecanicas'].max()
print(f'O maior número de categorias em um jogo é {max_number_categ}, e o de mecânicas é {max_number_mechanic}.')

O maior número de categorias em um jogo é 6, e o de mecânicas é 6.


Aqui temos uma decisão a fazer: quantas categorias e mecânicas entrarão para o grupo das que melhores representam os jogos populares?

No tópico "Análise de características" conseguimos as melhores categorias e mecânicas que compõem os jogos famosos, e cada uma tinha seu percentual de participação na soma total de aparições, então caso selecionarmos todas as características, a nossa análise ficará abrangente demais e não teremos como resultado jogos realmente pertinentes para trabalhar, então decidimos que não faz sentido selecionar categorias/mecânicas com percentuais de participações mais baixos e parecidos.

No caso das categorias, vemos que a partir da sétima os valores de participação começam a ficar menores e parecidos. Por conta disso, selecionamos as 6 melhores categorias para entrarem na definição do perfil de um jogo popular. Além disso, temos que o máximo de categorias que um jogo pode ter também é 6, o que concorda com a nossa escolha.

Para as mecânicas, vemos que 'Hand management' e 'Dice rolling' representam quase 18% do total de aparições, e que a partir delas duas, os valores diminuem e passam a variar menos. Nesse sentido, escolhemos as 4 maiores porcentagens de mecânicas para entrarem na definição do perfil de um jogo popular, essas 4 representam mais de 25% das aparições nos jogos famosos, de um total de 95 mecânicas.

Essas escolhas não foram arbitrárias, a idéia é somar a contagem de mecânicas e categorias de um jogo que são iguais as melhores selecionadas. Tendo isso em mente, vemos que a soma pode ir de 0 a 10, um intervalo fácil de conceber e trabalhar.

In [167]:
# Pegando as 6 melhores categorias e 4 melhores mecanicas
melhores_categorias = category_count2e4['feature'].head(6).copy()
melhores_mecanicas = mechanic_count2e4['feature'].head(4).copy()
melhores_familias = family_count2e4['feature'].head(10).copy()

In [168]:
# Funções de contagem
def conta_melhores_categorias(features: list) -> int:
    contador = 0
    for feature in features:
        if feature in melhores_categorias.to_list():
            contador += 1
    return contador

def conta_melhores_mecanicas(features: list) -> int:
    contador = 0
    for feature in features:
        if feature in melhores_mecanicas.to_list():
            contador += 1
    return contador

def conta_melhores_familias(features: list) -> int:
    contador = 0
    for feature in features:
        if feature in melhores_familias.to_list():
            contador += 1
    return contador

In [169]:
# Criando colunas de contagem,  e uma colunas de interesse, que é a soma das duas contagens.
df_filtrado['CMC'] = df_filtrado['boardgamecategory'].apply(conta_melhores_categorias)
df_filtrado['CMM'] = df_filtrado['boardgamemechanic'].apply(conta_melhores_mecanicas)
df_filtrado['CF'] = df_filtrado['boardgamefamily'].apply(conta_melhores_familias)

df_filtrado['interesse'] = df_filtrado['CMC'] + df_filtrado['CMM']

Criei as métricas de contagem a seguir:

CMC - Contagem que as melhores categorias aparecem (contagem melhores categorias)

CMM - Contagem que as melhores mecânicas aparecem (contagem melhores mecânicas)

CF - Contagem de famílias

Além desses indicadores de contagem, criei uma coluna chamada 'interesse', que é a soma da contagem de categorias e mecânicas. Com ela, podemos ter uma noção geral de quais jogos valem a pena serem divulgados, e também conseguimos uma métrica para filtro.

In [170]:
df_filtrado[['CMC', 'CMM', 'CF', 'interesse', 'usersrated', 'cluster', 'baverage', 'siteviews']].groupby(by=['cluster']).mean()

,CMC,CMM,CF,interesse,usersrated,baverage,siteviews
cluster,,,,,,,
0,0.616835,0.528374,0.197786,1.145210,222.489819,5.553143,3.276307e+04
1,1.225397,1.238095,0.806349,2.463492,10955.879365,7.011795,8.368732e+05
2,1.655172,1.586207,1.517241,3.241379,39694.172414,7.666282,3.858579e+06
3,0.866359,0.985517,0.574720,1.851876,2873.803818,6.332230,2.793917e+05
4,1.531646,1.392405,1.126582,2.924051,22617.531646,7.345284,1.745673e+06


In [171]:
df_filtrado['baverage2'] = (df_filtrado['baverage'] - min(df_filtrado['baverage'])) / (max(df_filtrado['baverage']) - min(df_filtrado['baverage']))
df_filtrado['boardgamehonor_cnt2'] = (df_filtrado['boardgamehonor_cnt'] - min(df_filtrado['boardgamehonor_cnt'])) / (max(df_filtrado['boardgamehonor_cnt']) - min(df_filtrado['boardgamehonor_cnt']))
df_filtrado['qualidade'] = df_filtrado['baverage2'] + df_filtrado['boardgamehonor_cnt2']
df_filtrado.head(1)

,object_id,name,yearpublished,sortindex,maxplayers,minplaytime,maxplaytime,minage,min_community,totalvotes,languagedependence,usersrated,average,baverage,stddev,avgweight,numweights,numgeeklists,numtrading,numwanting,numcomments,siteviews,numplays,numplays_month,news,blogs,weblink,podcast,boardgamedesigner_cnt,boardgameartist_cnt,boardgamepublisher_cnt,boardgamehonor_cnt,boardgamecategory_cnt,boardgamemechanic_cnt,boardgameexpansion_cnt,boardgameversion_cnt,boardgamefamily_cnt,boardgamecategory,boardgamemechanic,boardgamefamily,cluster,contagem de categorias,contagem de mecanicas,CMC,CMM,CF,interesse,baverage2,boardgamehonor_cnt2,qualidade
0,174430,Gloomhaven,2017,1,4,60,120,12,3.0,827.0,4,31254.0,8.85292,8.58424,1.59819,3.8078,1311,3657,313,1365,5972,8933078,230213,3478,7,471,31,139,1,3,9,23,5,12,4,19,7,"[Adventure, Exploration, Fantasy, Fighting, Mi...","[Campaign / Battle Card Driven, Cooperative Ga...","[Campaign Games, Components: Miniatures, Crowd...",2,5,6,4,2,4,6,0.982768,0.46,1.442768


In [172]:
df_filtrado['weblink2'] = (df_filtrado['weblink'] - min(df_filtrado['weblink'])) / (max(df_filtrado['weblink']) - min(df_filtrado['weblink']))
df_filtrado['blogs2'] = (df_filtrado['blogs'] - min(df_filtrado['blogs'])) / (max(df_filtrado['blogs']) - min(df_filtrado['blogs']))
df_filtrado['podcast2'] = (df_filtrado['podcast'] - min(df_filtrado['podcast'])) / (max(df_filtrado['podcast']) - min(df_filtrado['podcast']))
df_filtrado['news2'] = (df_filtrado['news'] - min(df_filtrado['news'])) / (max(df_filtrado['news']) - min(df_filtrado['news']))
df_filtrado['visibilidade_externa'] = df_filtrado['weblink2'] + df_filtrado['blogs2'] + df_filtrado['podcast2'] + df_filtrado['news2']

In [173]:
df_filtrado['usersrated2'] = (df_filtrado['usersrated'] - min(df_filtrado['usersrated'])) / (max(df_filtrado['usersrated']) - min(df_filtrado['usersrated']))
df_filtrado['numgeeklists2'] = (df_filtrado['numgeeklists'] - min(df_filtrado['numgeeklists'])) / (max(df_filtrado['numgeeklists']) - min(df_filtrado['numgeeklists']))
df_filtrado['numwanting2'] = (df_filtrado['numwanting'] - min(df_filtrado['numwanting'])) / (max(df_filtrado['numwanting']) - min(df_filtrado['numwanting']))
df_filtrado['numcomments2'] = (df_filtrado['numcomments'] - min(df_filtrado['numcomments'])) / (max(df_filtrado['numcomments']) - min(df_filtrado['numcomments']))
df_filtrado['siteviews2'] = (df_filtrado['siteviews'] - min(df_filtrado['siteviews'])) / (max(df_filtrado['siteviews']) - min(df_filtrado['siteviews']))
df_filtrado['visibilidade_interna'] = df_filtrado['usersrated2'] + df_filtrado['numgeeklists2'] + df_filtrado['numwanting2'] + df_filtrado['numcomments2'] + df_filtrado['siteviews2']

In [174]:
df_filtrado['visibilidade_geral'] = df_filtrado['visibilidade_externa'] + df_filtrado['visibilidade_interna']
df_filtrado.columns

Index(['object_id', 'name', 'yearpublished', 'sortindex', 'maxplayers',
       'minplaytime', 'maxplaytime', 'minage', 'min_community', 'totalvotes',
       'languagedependence', 'usersrated', 'average', 'baverage', 'stddev',
       'avgweight', 'numweights', 'numgeeklists', 'numtrading', 'numwanting',
       'numcomments', 'siteviews', 'numplays', 'numplays_month', 'news',
       'blogs', 'weblink', 'podcast', 'boardgamedesigner_cnt',
       'boardgameartist_cnt', 'boardgamepublisher_cnt', 'boardgamehonor_cnt',
       'boardgamecategory_cnt', 'boardgamemechanic_cnt',
       'boardgameexpansion_cnt', 'boardgameversion_cnt', 'boardgamefamily_cnt',
       'boardgamecategory', 'boardgamemechanic', 'boardgamefamily', 'cluster',
       'contagem de categorias', 'contagem de mecanicas', 'CMC', 'CMM', 'CF',
       'interesse', 'baverage2', 'boardgamehonor_cnt2', 'qualidade',
       'weblink2', 'blogs2', 'podcast2', 'news2', 'visibilidade_externa',
       'usersrated2', 'numgeeklists2', 'numwa

### Modelo preditivo

In [175]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [176]:
X = df.drop(['boardgameversion','playerage','average','object_id','name','totalvotes','usersrated','numgeeklists','numtrading','numwanting','numcomments','siteviews','numplays','numplays_month','news','blogs','weblink','podcast','baverage','stddev','boardgamehonor_cnt','boardgamedesigner','boardgameartist','boardgamepublisher','boardgamehonor','boardgamecategory','boardgamemechanic','boardgamefamily'], axis = 1)
for index, row in X.iterrows():
    index = pd.to_numeric(index)

    
y = df['qualidade']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

model = LinearRegression()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

r2 = r2_score(y_test, y_pred)

print(f'R^2 Score: {r2}')

plt.scatter(X_test['boardgameversion_cnt'], y_test, color='black', label='Observações')
plt.scatter(X_test['boardgameversion_cnt'], y_pred, color='blue', label='Previsões')
plt.legend()
plt.ylim(0, 10)
plt.show()

KeyError: "['boardgameversion', 'playerage'] not found in axis"

## Análise final

Com todos os dados coletados, podemos traçar o perfil de jogo popular, para assim verificar se existe algum jogo não tão popular que contém tal perfil. E por fim, mapearmos quais jogos são interessantes para serem recomendados aos clientes.

#### Características mapeadas:

Idade mínima: Entre 12 e 15 anos.

Categorias dos jogos populares: Fantasia, Luta, Aventura, Miniaturas, Economia e Jogos de carta.

Mecânicas dos jogos populares: Jogos que usam as mãos, Rolamento de dados, Diversos poderes dos jogadores e Jogos cooperativos.

Agora, vamos analisar todos os jogos que estão fora do cluster 2 e comparar suas características com as melhores selecionadas.

In [235]:
#criando as classes de idade,  mas agora para o data frame todo
for index, row in df_filtrado.iterrows():
    if row['minage'] <= 6:
        df_filtrado.loc[index,'age_class'] =  'early_child'
    if row['minage'] >= 7:
        df_filtrado.loc[index,'age_class'] =  'grown_child'
    if row['minage'] >= 12:
        df_filtrado.loc[index,'age_class'] =  'teenager'
    if row['minage'] >= 15:
        df_filtrado.loc[index,'age_class'] =  'adult' 

In [250]:
colunas_finais = ['object_id', 'age_class', 'cluster', 'name', 'yearpublished', 'usersrated', 'baverage', 'average', 'visibilidade_externa', 'visibilidade_interna'
                  , 'CMC', 'CMM', 'interesse', 'boardgamecategory', 'boardgamemechanic']
df_analise_final = df_filtrado[colunas_finais].copy()

Filtrando o DataFrame com apenas os jogos fora do cluster 2, taxa de interesse maior que 6, nota média dos jogadores maior que 5 e quantidade de avaliações menor que 740.
Esses valores não foram escolhidos a toa. A taxa de interesse mínima de 6 tem como justificativa a maior parte dos jogos do melhor cluster (2) possuirem essa taxa, a nota média de 5 é um parâmetro para não escolhermos jogos objetivamente ruins, a taxa de avaliação máxima de 740 representa a média dos valores dessa coluna. Essas duas ultimas mostram que queremos obter jogos que não tenham tanta visibilidade, já que jogos muito famosos já possuem diversas campanhas de marketing e divulgação.

In [252]:
tabela_interesse = df_analise_final.query(
    'cluster != 2 & interesse >= 6 & average >= 5 & usersrated <= 740 & age_class == "teenager"'
    ).sort_values(by=['interesse'], ascending=False).head(10)
tabela_interesse

,object_id,age_class,cluster,name,yearpublished,usersrated,baverage,average,visibilidade_externa,visibilidade_interna,CMC,CMM,interesse,boardgamecategory,boardgamemechanic
1464,218421,teenager,3,Street Masters,2018,568.0,6.25878,8.442100,0.077250,0.118344,5,3,8,"[Adventure, Card Game, Dice, Fantasy, Fighting...","[Cooperative Game, Dice Rolling, Grid Movement..."
3159,271060,teenager,0,Pathfinder Adventure Card Game Core Set,2019,270.0,5.84069,6.201231,0.047549,0.041220,4,4,8,"[Adventure, Card Game, Fantasy, Fighting]","[Cooperative Game, Deck, Dice Rolling, Hand Ma..."
10418,251661,teenager,3,Oathsworn Into the Deepwood,2020,57.0,5.53913,8.289470,0.020273,0.041243,5,3,8,"[Adventure, Card Game, Fantasy, Fighting, Matu...","[Action Points, Cooperative Game, Deck, Dice R..."
7067,192315,teenager,0,Widower's Wood An Iron Kingdoms Adventure Boa...,2017,79.0,5.59059,7.319620,0.035924,0.021575,4,4,8,"[Adventure, Exploration, Fantasy, Fighting, Mi...","[Cooperative Game, Dice Rolling, Grid Movement..."
9148,217012,teenager,0,Alchemical Crystal Quest (second edition),2017,89.0,5.55354,6.968990,0.001727,0.009294,4,3,7,"[Adventure, Fantasy, Fighting, Miniatures]","[Cooperative Game, Dice Rolling, Modular Board..."
2592,212346,teenager,3,Shadows of Brimstone Forbidden Fortress,2018,306.0,5.93614,8.665600,0.059240,0.069914,4,3,7,"[Adventure, Exploration, Fantasy, Fighting, Ho...","[Cooperative Game, Dice Rolling, Grid Movement..."
3856,175334,teenager,0,The Undercity An Iron Kingdoms Adventure Boar...,2015,301.0,5.75861,7.150400,0.084428,0.054927,4,3,7,"[Adventure, Exploration, Fantasy, Fighting, Mi...","[Cooperative Game, Dice Rolling, Grid Movement..."
2474,190400,teenager,0,Aventuria Adventure Card Game,2016,463.0,5.96367,7.564150,0.064007,0.061333,4,3,7,"[Adventure, Card Game, Fantasy, Fighting]","[Cooperative Game, Deck, Dice Rolling, Hand Ma..."
12931,262733,teenager,0,Reichbusters Projekt Vril,2019,50.0,5.51813,8.380000,0.008171,0.019465,3,3,6,"[Adventure, Fighting, Miniatures, World War II...","[Area Movement, Cooperative Game, Dice Rolling..."
18014,150811,teenager,0,Mega Man The Board Game,2016,247.0,5.42204,5.289310,0.066036,0.034743,4,2,6,"[Adventure, Card Game, Fighting, Miniatures, M...","[Action Points, Deck, Hand Management, Point t..."


Veja que os jogos acima não são nada populares, tanto a visibilidade externa quanto interna, mas possuem nota de BGG e jogadores relativamente boas, e poucas avaliações. Além disso, sua nota de interesse é sempre 6 ou maior, o que reflete a similaridade entre suas categorias e mecânicas com as melhores selecionadas.

Eles são, provavelmente, fruto de uma campanha de marking de desenvolvedora ruim, de um recente lançamento, ou até mesmo são ofuscado por jogos mais populares mas da mesma família, como Pathfinder core set é ofuscado pela sua versão padrão, mas poderia atingir um público que não quer gastar tanto com um jogo tão grande e complexo, e optaria por uma versão mais sucinta mas de igual qualidade.

In [253]:
for nome in tabela_interesse['name']:
    print(nome)

Street Masters
Pathfinder Adventure Card Game  Core Set
Oathsworn  Into the Deepwood
Widower's Wood  An Iron Kingdoms Adventure Board Game
Alchemical Crystal Quest (second edition)
Shadows of Brimstone  Forbidden Fortress
The Undercity  An Iron Kingdoms Adventure Board Game
Aventuria  Adventure Card Game
Reichbusters  Projekt Vril
Mega Man  The Board Game


In [ ]:
# Aqui o próprio cliente pode filtrar qual categoria ele quer que os jogos filtrados acima contenham, basta mudar o atributo 'Adventure'
tabela_interesse.loc[[index for index, row in tabela_interesse.iterrows() if "Adventure" in row['boardgamecategory'] and 'Cooperative Game' in row['boardgamemechanic']]]

,object_id,cluster,name,yearpublished,usersrated,baverage,average,siteviews,news,blogs,weblink,podcast,CMC,CMM,CF,interesse,boardgamecategory,boardgamemechanic
7067,192315,0,Widower's Wood An Iron Kingdoms Adventure Boa...,2017,79.0,5.59059,7.31962,46909,1,3,0,0,4,4,1,8,"[Adventure, Exploration, Fantasy, Fighting, Mi...","[Cooperative Game, Dice Rolling, Grid Movement..."
9148,217012,0,Alchemical Crystal Quest (second edition),2017,89.0,5.55354,6.96899,18921,0,2,0,0,4,3,1,7,"[Adventure, Fantasy, Fighting, Miniatures]","[Cooperative Game, Dice Rolling, Modular Board..."
9011,272533,0,Kingdom Rush Rift in Time,2020,41.0,5.55538,7.94927,42898,0,15,0,0,4,3,1,7,"[Adventure, Fantasy, Fighting, Medieval, Minia...","[Cooperative Game, Hand Management, Modular Bo..."
3693,174805,0,Champions of Hara,2018,236.0,5.77643,7.64839,65029,2,14,5,4,4,2,1,6,"[Adventure, Fantasy, Fighting, Miniatures, Puz...","[Area Movement, Cooperative Game, Grid Movemen..."
10737,284751,0,Armata Strigoi,2019,37.0,5.53629,7.44595,17049,1,1,3,0,4,2,0,6,"[Adventure, Fantasy, Fighting, Miniatures]","[Cooperative Game, Deck, Hand Management, Modu..."
10744,158487,0,Dragon Tides,2015,71.0,5.53624,6.42958,57357,1,3,1,1,4,2,1,6,"[Adventure, Bluffing, Card Game, Dice, Fightin...","[Action Points, Card Drafting, Cooperative Gam..."
11187,197928,0,Dreamwars,2017,74.0,5.53241,6.23514,27096,0,6,0,2,3,3,1,6,"[Adventure, Fantasy, Horror, Miniatures, Scien...","[Action Points, Cooperative Game, Dice Rolling..."
